In [1]:
from neo4j import GraphDatabase
import pandas as pd
from openai import OpenAI
import numpy as np
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import json
from collections import Counter
import mysql.connector
import os
import re
from dotenv import load_dotenv
load_dotenv()
from sklearn.preprocessing import MinMaxScaler
import warnings
import logging

warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger("neo4j").setLevel(logging.ERROR)

In [2]:
# DB接続処理
# Neo4jに接続
uri = "bolt://localhost:7687"
user = "neo4j"
password = "abcd7890"

# ドライバを作成
driver = GraphDatabase.driver(uri, auth=(user, password))

In [3]:
# DB接続処理
# MySQLに接続
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="password",
    database="foodb"
)

In [4]:
# ChatGPTを使うのでAPIキーを設定
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
gpt_client = OpenAI(
    api_key=OPENAI_API_KEY
)

In [5]:
# ChatGPTを使って意味のある文章を生成する
def get_gpt_result(prompt):
    response = gpt_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()


In [6]:
# Word2Vecモデルを学習
#model = Word2Vec(sentences=all_flavors, vector_size=100, window=5, min_count=1, sg=1)
model = Word2Vec.load("../data/fine_tuned_word2vec.model")

# フレーバーのベクトルを確認
green_vector = model.wv['green']
print(f'{model.vector_size}, {len(green_vector)}')

# similar to の結果をjsonで返す
def similar_to_json(word):
    try:    
        similar_words = model.wv.most_similar(positive=[word])
        similar_words_dict = {word: score for word, score in similar_words}
        json_data = json.dumps(similar_words_dict, indent=4)
        return json_data
    except KeyError:
        return json.dumps({})

# vectorを返す
def get_word_vector(word):
    try:    
        return model.wv[word]
    except KeyError:
        print("error key: ", word)
        return [0.0] * model.vector_size 
        


100, 100


In [7]:
# テキストフォーマットの指定
def format_text(text):
    if text is None:
        return ""
    text = text.replace('(', '_')
    text = text.replace(')', '_')
    text = text.replace("/", '_')
    text = text.replace(";", '_')
    text = text.replace(":", '_')
    text = text.replace("&", '_')
    text = text.replace("[", '')
    text = text.replace("]", '')    
    text = text.replace('"', '')
    text = text.replace('<', '')
    text = text.replace('>', '')
    text = text.replace(', ', ',')
    text = text.replace('.', '')
    text = text.replace(',', '_')
    text = text.replace('-', ' ')
    text = text.replace(' ', '_')
    text = text.replace('\n', '')
    text = text.replace('%', '')
    text = text.replace("'", '')
    text = text.replace("/t", '')
    text = text.replace("\\", '')
    text = text.replace("é", '')
    text = text.replace("ç", '')
    text = text.replace("+", '')
    text = text.replace("`", '')
    text = text.replace("`", '')
    text = text.replace("`", '')
    text = text.replace("`", '')
    text = text.replace("ã", '')
    text = text.replace("â", '')
    text = text.replace("ƒ", '')
    text = text.replace("€", '')
    text = text.replace('”', '')
    text = text.replace('`', '')
    text = text.replace('!', '')
    text = text.replace('|', '')
    text = text.replace("\n", "")
    text = text.replace("\r", "")
    text = re.sub(r'\s+', '', text)
    text = text.lower()
    return text

def escape_sql_string(text):
    text = text.rstrip("\\")
    return text.replace("'", "''")

In [8]:
# Foodとカテゴリのパッチファイルを読み込む
pd_category_patch = pd.read_csv("../data/CategoryPatch.csv")
pd_food_patch = pd.read_csv("../data/FoodPatch.csv")
pd_aroma_patch = pd.read_csv("../data/AromaPatch.csv")
pd_category_patch = pd.read_csv("../data/CategoryPatch.csv")
pd_category_sub_patch = pd.read_csv("../data/CategorySubPatch.csv")

## 以下、neo4jへのデータ投入ロジック
#### 時間がかかるので注意

In [9]:
#
# GroupとSubGroupを初期化
#
# 古いデータを削除してリセット
def initialize_groups(tx):
    # Molecule ノードの存在を確認
    tx.run("MATCH (f:FoodGroup) DETACH DELETE f;")
    tx.run("MATCH (f:FoodSubGroup) DETACH DELETE f;")

# 新規レコードを作成
def create_groups(tx, node, group_name):
    if group_name is None or group_name == "":
        return
    
    vec_key = format_text(group_name)
    most_similar = similar_to_json(vec_key)
    word_vector = get_word_vector(vec_key)
            
    query = f"""
        MERGE (m:{node} {{
            id: $id,
            name: $name,
            most_similar_json: $most_similar_json,
            word_vector: $word_vector
        }})
        """
    
    tx.run(query,
        id=vec_key,
        name=group_name,
        most_similar_json=most_similar,
        word_vector=word_vector)
    
# カテゴリのパッチの情報取得
def find_category_patch_row(df, vec_key):
    df_row = df[df["id"] == vec_key]
    if len(df_row) > 0:
        return df_row["display_name_ja"].values[0], df_row["class_name"].values[0]
    return "", ""

 # グループとサブグループの関連付け
def create_group_connections(tx, group_name, subgroup_name):
    vec_key = format_text(group_name)
    vec_key_sub = format_text(subgroup_name)
    jp_name, class_name = find_category_patch_row(pd_category_patch, vec_key)
    sub_jp_name, sub_class_name = find_category_patch_row(pd_category_sub_patch, vec_key_sub)
            
    tx.run("""
        MATCH (f:FoodGroup {id: $group_name})
        MATCH (s:FoodSubGroup {id: $subgroup_name})
        MERGE (f)-[:CONTAINS]->(s)
        SET f.display_name_ja = $jp_name
        SET f.class_name = $class_name
        SET s.display_name_ja = $sub_jp_name
        SET s.class_name = $sub_class_name
        """,
        group_name=vec_key,
        subgroup_name=vec_key_sub,
        jp_name=jp_name,
        class_name=class_name,
        sub_jp_name=sub_jp_name,
        sub_class_name=sub_class_name)

# インデックス追加
def append_group_index(tx):
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (f:FoodGroup) REQUIRE f.id IS UNIQUE")
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (f:FoodSubGroup) REQUIRE f.id IS UNIQUE")

# １行つづ検索してデータを投入
with driver.session() as session:
    session.execute_write(initialize_groups)
    
    query = "select food_group, food_subgroup from foods where food_group is not null group by food_group, food_subgroup"
    cursor1 = connection.cursor(dictionary=True)
    cursor1.execute(query)
    for row in cursor1.fetchall():
        session.execute_write(create_groups, "FoodGroup", row["food_group"])
        session.execute_write(create_groups, "FoodSubGroup", row["food_subgroup"])
        session.execute_write(create_group_connections, row["food_group"], row["food_subgroup"])
    
    session.execute_write(append_group_index)

In [10]:
pd_food_patch.tail()

,key,original_name,word_vector_key,display_name_ja,category_en,search_query,category_ja,origin_name,origin_name_ja
987,sablefish,Sablefish,NaN,サブレフィッシュ,Fish,NaN,魚,Ocean,海
988,taco,Taco,NaN,タコ,Cooking,NaN,料理,NaN,その他
989,taco_shell,Taco shell,NaN,タコシェル,Cooking,NaN,料理,NaN,その他
990,tostada,Tostada,NaN,トスターダ,Cooking,NaN,料理,NaN,その他
991,waffle,Waffle,NaN,ワッフル,Sweets,NaN,菓子,Field,畑


In [11]:
#
# Foodを初期化
#
# 古いデータを削除してリセット
# データ操作を行うトランザクション
def drop_food_data(tx):
    tx.run("MATCH (f:Food) DETACH DELETE f;")
    tx.run("MATCH (f:FoodSubType) DETACH DELETE f;")
    
# 新規レコードを作成
def create_foods(tx, food_name, food_name_scientific, food_name_subs):
    vec_key = format_text(food_name)
    most_similar = similar_to_json(vec_key)
    word_vector = get_word_vector(vec_key)
    
    # 単語の補正パッチを当てる
    pd_food_patch_row = pd_food_patch[pd_food_patch["key"] == vec_key]
    if len(pd_food_patch_row) > 0:
        patch_name = pd_food_patch_row["word_vector_key"].values[0]
        if not pd.isnull(patch_name):
            patch_key = format_text(patch_name)
            most_similar = similar_to_json(patch_key)
            word_vector = get_word_vector(patch_key)
            food_name = patch_name
            
    search_query = " ".join([food_name or "", food_name_scientific or "", food_name_subs or "",
                               str(pd_food_patch_row["search_query"].iloc[0] if not pd_food_patch_row["search_query"].empty else ""),
                                   str(pd_food_patch_row["display_name_ja"].iloc[0] if not pd_food_patch_row["display_name_ja"].empty else "")])
    
    find_query = """
        MATCH (fs:Food {id: $id}) RETURN fs;
     """
    result = tx.run(find_query, id=vec_key).single()
    if result is None:
        display_name_ja = pd_food_patch_row["display_name_ja"].values
        display_name_ja = display_name_ja[0] if len(display_name_ja) > 0 else ""
        query = f"""
            MERGE (m:Food {{
                id: $id,
                name: $name,
                search_query: $search_query,
                most_similar_json: $most_similar_json,
                word_vector: $word_vector,
                display_name_ja: $display_name_ja
            }})
            """
        
        tx.run(query,
            id=vec_key,
            name=food_name,
            search_query=search_query,
            most_similar_json=most_similar,
            word_vector=word_vector,
            display_name_ja=display_name_ja
        )
        
    
# 新規レコードを作成
def create_food_sub_type(tx, food_name, food_sub_type_name):
    vec_key = format_text(food_sub_type_name)
    most_similar = similar_to_json(vec_key)
    search_query = " ".join([food_sub_type_name or ""])
    word_vector = get_word_vector(vec_key)
            
    find_query = """
        MATCH (fs:FoodSubType {id: $id}) RETURN fs;
     """
    result = tx.run(find_query, id=vec_key).single()
    if result is None:    
        query = f"""
            MERGE (m:FoodSubType {{
                id: $id,
                name: $name,
                search_query: $search_query,
                most_similar_json: $most_similar_json,
                word_vector: $word_vector
            }})
            """
        
        tx.run(query,
            id=vec_key,
            name=food_sub_type_name,
            search_query=search_query,
            most_similar_json=most_similar,
            word_vector=word_vector)

# 新規レコードを作成
def connect_food_group(tx, food_name, food_group, food_sub_group):
    food_vec_key = format_text(food_name)
    group_vec_key = format_text(food_group)
    groupsub_vec_key = format_text(food_sub_group)

    query = f"""
        MATCH (f:Food {{id: $food_id}})
        MATCH (fg:FoodGroup {{id: $food_group}})
        MATCH (fsg:FoodSubGroup {{id: $food_subgroup}})
        MERGE (f)-[:BELONGS_TO]->(fg)
        MERGE (f)-[:BELONGS_TO]->(fsg)
        SET f.group = $food_group
        SET f.sub_group = $food_subgroup
        """
    
    tx.run(query,
        food_id=food_vec_key,
        food_group=group_vec_key,
        food_subgroup=groupsub_vec_key)

# インデックス追加
def append_foods_index(tx):
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (f:Food) REQUIRE f.id IS UNIQUE")
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (f:FoodSubType) REQUIRE f.id IS UNIQUE")
    
# １行つづ検索してデータを投入
with driver.session() as session:
    session.execute_write(drop_food_data)

    query = """
    select foods.name as food_name, foods.name_scientific as food_name_scientific, contents.orig_food_common_name
    orig_food_common_name from foods inner join contents on foods.id = contents.food_id group by foods.name,contents.orig_food_common_name
    """
    cursor1 = connection.cursor(dictionary=True)
    cursor1.execute(query)
    for row in cursor1.fetchall():
        session.execute_write(create_foods, row["food_name"], row["food_name_scientific"], row["orig_food_common_name"])
    
    query2 = """
    select foods.name as food_name, foods.name_scientific as food_name_scientific, foods.food_group, foods.food_subgroup,
    contents.orig_food_common_name as orig_food_common_name from foods left join contents on foods.id = contents.food_id 
    group by foods.name,contents.orig_food_common_name
    """
    cursor2 = connection.cursor(dictionary=True)
    cursor2.execute(query2)
    for row in cursor2.fetchall():
        if row["orig_food_common_name"] is None:
            row["orig_food_common_name"] = row["food_name"]
        
        session.execute_write(create_food_sub_type, row["food_name"], row["orig_food_common_name"])
        session.execute_write(connect_food_group, row["food_name"], row["food_group"], row["food_subgroup"])
        
    session.execute_write(append_foods_index)

error key:  mandarin_orange
error key:  mandarin_orange
error key:  mandarin_orange
error key:  mandarin_orange
error key:  mandarin_orange
error key:  mandarin_orange
error key:  mandarin_orange
error key:  mandarin_orange
error key:  mandarin_orange
error key:  mandarin_orange
error key:  mandarin_orange
error key:  mandarin_orange
error key:  mandarin_orange
error key:  mandarin_orange
error key:  mandarin_orange
error key:  mandarin_orange
error key:  sunburst_squash
error key:  sunburst_squash
error key:  sunburst_squash
error key:  sunburst_squash
error key:  sunburst_squash
error key:  sunburst_squash
error key:  sunburst_squash
error key:  sunburst_squash
error key:  sunburst_squash
error key:  sunburst_squash
error key:  sunburst_squash
error key:  mandarin_orange
error key:  mandarin_orange
error key:  mandarin_orange
error key:  sunburst_squash
error key:  drage_average_values
error key:  drage_liquorice
error key:  monterey_jack_cheese
error key:  wine_red_ros_non_alcoholic

In [12]:
# FoodとFoodSubTypeの関連付け
def create_food_connections(tx, food_name, food_sub_type_name):
    food_id = format_text(food_name)
    food_sub_id = format_text(food_sub_type_name)
    find_query = """
        MATCH (f:Food {id: $food_id})-[r:HAS_SUBTYPE]->(fs:FoodSubType {id: $food_sub_id}) RETURN r;
     """
    result = tx.run(find_query, food_id=food_id, food_sub_id=food_sub_id).single()
    if result is None:
        try:
            
            tx.run("""
                MATCH (f:Food {id: $food_id})
                MATCH (s:FoodSubType {id: $food_sub_type_id})
                MERGE (f)-[:HAS_SUBTYPE]->(s)
                """,
                food_id=food_id,
                food_sub_type_id=food_sub_id)
        except Exception as e:
            print("error: ", food_id, food_sub_id)
            print(e)
            
    
# グループとフードの関連付け
def create_food_group_connections(tx, food_name, group_name, subgroup_name):
    tx.run("""
        MATCH (f:Food {id: $food_id})
        MATCH (g:FoodGroup {id: $group_id})
        MERGE (f)-[:HAS_GROUP]->(g)
        """,
        food_id=format_text(food_name),
        group_id=format_text(group_name))

    tx.run("""
        MATCH (f:Food {id: $food_id})
        MATCH (g:FoodSubGroup {id: $sub_group_id})
        MERGE (f)-[:HAS_SUB_GROUP]->(g)
        """,
        food_id=format_text(food_name),
        sub_group_id=format_text(subgroup_name))
    

# １行つづ検索してデータを投入
with driver.session() as session:
    query = """
    select foods.name as food_name, foods.name_scientific as food_name_scientific, foods.food_group, foods.food_subgroup,
    orig_food_common_name from foods left join contents on foods.id = contents.food_id group by foods.name,contents.orig_food_common_name
    """
    cursor1 = connection.cursor(dictionary=True)
    cursor1.execute(query)
    for row in cursor1.fetchall():
        if row["orig_food_common_name"] is None:
            row["orig_food_common_name"] = row["food_name"]
        
        session.execute_write(create_food_connections, row["food_name"], row["orig_food_common_name"])
        session.execute_write(create_food_group_connections, row["food_name"], row["food_group"], row["food_subgroup"])
    
    session.execute_write(append_foods_index)

In [13]:
#
# Compoundを初期化
#
# 古いデータを削除してリセット
def initialize_compound(tx):
    # Molecule ノードの存在を確認
    tx.run("MATCH (f:Compound) DETACH DELETE f;")

# 新規レコードを作成
def create_compound(tx, compound_name):
    if compound_name is None or compound_name == "":
        return
    
    vec_key = format_text(compound_name)
    #most_similar = similar_to_json(vec_key)
    #word_vector = get_word_vector(vec_key)

    query = """
        MERGE (m:Compound {id: $id})
        ON CREATE SET m.name = $name
        ON MATCH SET m.name = $name
        """
    
    tx.run(query,
        id=vec_key,
        name=compound_name)

# インデックス追加
def append_compound_index(tx):
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (c:Compound) REQUIRE c.id IS UNIQUE")

# １行つづ検索してデータを投入
with driver.session() as session:
    session.execute_write(initialize_compound)
    
    query = "select orig_source_name from contents group by orig_source_name"
    cursor1 = connection.cursor(dictionary=True)
    cursor1.execute(query)
    for row in cursor1.fetchall():
        session.execute_write(create_compound, row["orig_source_name"])
    
    session.execute_write(append_compound_index)

In [14]:
#
# Aroma初期化
#
# 古いデータを削除してリセット
def initialize_aroma(tx):
    # Molecule ノードの存在を確認
    tx.run("MATCH (f:Aroma) DETACH DELETE f;")

# 新規レコードを作成
def create_aroma(tx, aroma_name):
    if aroma_name is None or aroma_name == "":
        return
    
    vec_key = format_text(aroma_name)
    most_similar = similar_to_json(vec_key)
    word_vector = get_word_vector(vec_key)
    
    display_name_ja = "-"
    color_code = "#ffffff"
    pd_patch_row = pd_aroma_patch[pd_aroma_patch["vec_key"] == vec_key]
    if color_code is not None and len(pd_patch_row) > 0:
        display_name_ja = pd_patch_row["display_name_ja"].values[0]
        color_code = pd_patch_row["color_code"].values[0]
    
    query = """
        MERGE (m:Aroma {
            id: $id,
            name: $name,
            most_similar_json: $most_similar_json,
            word_vector: $word_vector,
            display_name_ja: $display_name_ja,
            color: $color
        })
        """
    
    tx.run(query,
        id=vec_key,
        name=aroma_name,
        most_similar_json=most_similar,
        word_vector=word_vector,
        display_name_ja=display_name_ja,
        color=color_code
    )

# インデックス追加
def append_aroma_index(tx):
    tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (f:Aroma) REQUIRE f.id IS UNIQUE")

# １行つづ検索してデータを投入
with driver.session() as session:
    session.execute_write(initialize_aroma)
    
    query = "select name from flavors group by name"
    cursor1 = connection.cursor(dictionary=True)
    cursor1.execute(query)
    for row in cursor1.fetchall():
        session.execute_write(create_aroma, row["name"])
    
    session.execute_write(append_aroma_index)

In [15]:
# aromaの一覧
cursor1 = connection.cursor(dictionary=True)
cursor1.execute("select name from flavors group by name")
flavor_key_list = []
for row in cursor1.fetchall():
        flavor_key_list.append(format_text(row["name"]))
flavor_pd = pd.DataFrame(columns=flavor_key_list)
flavor_pd

,4_t_butyl_cyclohexyl_acetate,absinthe,absolute,acacia,acetate,acetic,acetoin,acetone,acetophenone,acid,...,wine,wine_like,wine_lee,winey,wintergreen,wood,woody,yeast,yeasty,ylang


In [16]:
#
# CompoundをFoodSubTypeと紐づける
# 
# mg/100gのデータを取得
scaler = MinMaxScaler()

def apply_calculation(rows):
    matrix_df = flavor_pd.copy()

    df = pd.DataFrame(rows)    
    df = df[df["flavor_name"] != "odorless"]
    if "standard_content" in df.columns:
        df["standard_content"] = pd.to_numeric(df["standard_content"], errors="coerce").fillna(0)
    
    if len(df) > 0:
        df["standard_content"] = scaler.fit_transform(df[["standard_content"]].values)

    compound_sum = df.groupby("compound_id")["standard_content"].sum().apply(lambda x: round(x, 5)).reset_index()
    flavor_sum = df.groupby("flavor_name")["standard_content"].sum().apply(lambda x: round(x, 5)).reset_index()
        
    for index, row in flavor_sum.iterrows():
        aroma_id = format_text(row["flavor_name"])
        matrix_df.loc[0, aroma_id] = float(row["standard_content"])

    matrix_df = matrix_df.fillna(0).astype(float)
    compound_sum["ratio"] = compound_sum["standard_content"] / compound_sum["standard_content"].sum()
    flavor_sum["ratio"] = flavor_sum["standard_content"] / flavor_sum["standard_content"].sum()

    return df, compound_sum, flavor_sum, matrix_df

def make_connection(tx, info, rows):
    food_sub_type_id = format_text(info["orig_food_common_name"])
    df, df_compound, df_aroma, matrix_df = apply_calculation(rows)
    max_flavor_name = ""
    # TODO: pandasのdataframeをjsonにしたい
    
    flavor_json = df_aroma.to_json(orient="records") 
    # flavor構成要素をvector化して入れる
    if matrix_df.empty:
        matrix_value = np.zeros(len(flavor_pd.columns))
    else:
        matrix_value = matrix_df.values.tolist()[0]
        max_flavor_name = df_aroma.loc[df_aroma["standard_content"].idxmax(), "flavor_name"]

    tx.run("""
        MATCH (f:FoodSubType {id: $food_sub_type_id}) 
        SET f.flavor_vector = $flavor_vector
        SET f.flavor_json = $flavor_json
        """,
        food_sub_type_id=food_sub_type_id,
        flavor_vector=matrix_value,
        flavor_json=flavor_json
    )
    
    for _index, row0 in df.iterrows():
        food_id = format_text(row0["food_name"])
        aroma_id = format_text(row0["flavor_name"])
        
        tx.run("""
            MATCH (c:Compound {id: $compound_id})
            MATCH (f:Aroma {id: $aroma_id})
            MERGE (c)-[r:SCENTED]->(f)
            """,
            compound_id=row0["compound_id"],
            aroma_id=aroma_id,
            )
    
    for _index, row1 in df_compound.iterrows():
        tx.run("""
            MATCH (f:FoodSubType {id: $food_sub_type_id}) 
            MATCH (c:Compound {id: $compound_id}) 
            MERGE (f)-[r:CONTAINS]->(c)
            SET r.ratio = $ratio
            """,
            food_sub_type_id=food_sub_type_id, 
            compound_id=format_text(row0["compound_name"]),
            ratio=float(row1["ratio"])
            )

    for _index, row2 in df_aroma.iterrows():
        aroma_id = format_text(row2["flavor_name"])

        tx.run("""
            MATCH (f:Food {id: $food_id})
            MATCH (fs:FoodSubType {id: $food_sub_type_id}) 
            MATCH (a:Aroma {id: $aroma_id}) 
            MERGE (f)-[r:SCENTED]->(a)
            MERGE (fs)-[r2:SCENTED]->(a)
            SET r.ratio = $ratio
            SET r2.ratio = $ratio
            """,
            food_id=food_id,
            food_sub_type_id=food_sub_type_id, 
            aroma_id=aroma_id,
            ratio=float(row2["ratio"]))

    if max_flavor_name != "":
        tx.run("""
            MATCH (f:FoodSubType {id: $food_sub_type_id}) 
            MATCH (a:Aroma {id: $aroma_id}) 
            MERGE (f)-[r:KEY_NOTE]->(a)
            SET f.key_note = $aroma_id
            """,
            food_sub_type_id=food_sub_type_id, 
            aroma_id=max_flavor_name)


sql_outer = "select contents.food_id, contents.orig_food_common_name from foods inner join contents on foods.id = contents.food_id where citation_type ='DATABASE' and source_type = 'Compound' and standard_content > 0.0 group by food_id, orig_food_common_name"

 # １行つづ検索してデータを投入
with driver.session() as session:
    cursor1 = connection.cursor(dictionary=True)
    cursor1.execute(sql_outer)
    for row in cursor1.fetchall():
        if row['orig_food_common_name'] is None or row['orig_food_common_name'] == "":
            continue
        
        sql_inner = f"""
            SELECT foods.id, foods.name as food_name, orig_food_common_name,
            contents.orig_source_name, IFNULL(contents.standard_content, 0.0) AS standard_content, contents.orig_unit, compounds.id as compound_id, compounds.name as compound_name,
            flavors.id, flavors.name as flavor_name
            FROM 
            foods INNER JOIN contents ON foods.id = contents.food_id
            INNER JOIN compounds ON contents.source_id = compounds.id 
            LEFT JOIN compounds_flavors on compounds.id = compounds_flavors.compound_id
            LEFT JOIN flavors on compounds_flavors.flavor_id = flavors.id
            where contents.citation_type ='DATABASE' and contents.source_type = 'Compound' and contents.standard_content > 0.0 and contents.food_id={row['food_id']} 
            and orig_food_common_name ='{escape_sql_string(row['orig_food_common_name'])}' and flavors.name is not null
            order by contents.standard_content desc limit 30;
            """
        cursor2 = connection.cursor(dictionary=True)
        try:
            cursor2.execute(sql_inner)
            inner_rows = cursor2.fetchall()
        except:
            print("error", sql_inner)
            continue
        
        if not inner_rows:
            #print("error: ", row)
            continue
        
        session.execute_write(make_connection, row, inner_rows)
    

In [17]:
#
# Word2Vecのネットワークを構築する
# 
# FoodSubTypeが300なので
batch_size = 5000  # 推奨値

def apply_word2_vec_network_batch(tx, offset, limit):
    query = """
      MATCH (a:FoodSubType), (b:FoodSubType)
      WHERE id(a) < id(b)
        AND a.word_vector IS NOT NULL
        AND b.word_vector IS NOT NULL
      WITH a, b, gds.similarity.cosine(a.word_vector, b.word_vector) AS similarity
      WHERE similarity > 0.7
      SKIP $offset LIMIT $limit
      MERGE (a)-[r:SIMILAR {weight: similarity}]->(b);
    """
    tx.run(query, offset=offset, limit=limit)

offset = 0
with driver.session() as session:
    while offset < 50000:
        session.execute_write(apply_word2_vec_network_batch, offset, batch_size)
        offset += batch_size


In [18]:
#
# FoodSubでFoodを代表するもののflavor_vectorをFoodにいれる
# 
default_vec = np.zeros(len(flavor_pd.columns))
fetch_all_food_query = """
MATCH (f:Food) RETURN f
"""

find_query = """
// Food と FoodSubType のペアを取得して類似度を計算
MATCH (f:Food {id: $food_id})-[:HAS_SUBTYPE]->(fs:FoodSubType)
WHERE f.word_vector IS NOT NULL 
  AND fs.word_vector IS NOT NULL 
  AND fs.flavor_vector IS NOT NULL
WITH f, fs, gds.similarity.cosine(f.word_vector, fs.word_vector) AS similarity
ORDER BY similarity DESC
LIMIT 1
RETURN f.name AS foodName, fs AS food_sub, similarity
"""

with driver.session() as session:
    result = session.run(fetch_all_food_query)
    for record in result:
        food_id = record["f"]["id"]
        result = session.run(find_query, food_id=food_id)
        data = result.data()
        if data is None or len(data) == 0:
            session.run("""
                MATCH (f:Food {id: $food_id})
                SET f.flavor_vector = $default_vec
                """,
                food_id=food_id,
                default_vec=default_vec
            )
        else:
            food_sub_id = data[0]["food_sub"]["id"]
            session.run("""
                MATCH (f:Food {id: $food_id})
                MATCH (fs:FoodSubType {id: $food_sub_id})
                MERGE (f)-[:REPRESENT]->(fs)
                SET f.flavor_vector = $flavor_vector
                """,
                food_id=food_id,
                food_sub_id=food_sub_id,
                flavor_vector=data[0]["food_sub"]["flavor_vector"]
            )

In [19]:
# フルテキストインデックスを作成
with driver.session() as session:
    session.run("DROP INDEX food_origin_index_text_search IF EXISTS")
    session.run("DROP INDEX food_sub_index_text_search IF EXISTS")
    session.run("CREATE FULLTEXT INDEX food_origin_index_text_search IF NOT EXISTS FOR (f:Food) ON EACH [f.search_query]")
    session.run("CREATE FULLTEXT INDEX food_sub_index_text_search IF NOT EXISTS FOR (fs:FoodSubType) ON EACH [fs.search_query]")


### Recipe JSONによるネットワーク構築

In [20]:
new_json_data = {}
with open("../data/formatted_json_recipe.json", "r") as file:
    new_json_data = json.load(file)
print(len(new_json_data))

1322


In [21]:
def find_food(tx, search_term):
    find_food_query = """
    CALL db.index.fulltext.queryNodes("food_origin_index_text_search", $search_term)
    YIELD node, score as full_text_score
    ORDER BY full_text_score DESC, size(node.name)
    LIMIT 1
    RETURN node.id as node_id, node.name as node_name, node.flavor_vector as flavor_vector, node.word_vector as word_vector, full_text_score    
    """
    result = tx.run(find_food_query, search_term=search_term).data()
    if result:
        return result[0]
        
    return None

In [22]:
def safe_value(value, default):
    return value if value is not None else default

def drop_recipe_data(tx):
    tx.run("MATCH (r:Recipe) DETACH DELETE r;")
    
def make_recipe(session, title, recipe):
    recipe_id = format_text(title)
    session.run("""
        MERGE (r:Recipe {id: $recipe_id})
        SET r.title = $title
        SET r.rating = $rating
        SET r.reviews = $reviews
        SET r.total_time = $total_time
        """,
        recipe_id=recipe_id,
        title=title,
        rating=safe_value(recipe["rating"], 0),
        reviews=safe_value(recipe["reviews"], 0),
        total_time=safe_value(recipe["total_time"], 0)
    )
    return recipe_id
    
def make_food_network(session, recipe_id, food1, food2):
    if food1 is not None and food2 is not None:
        session.run("""
            MATCH (r:Recipe {id: $recipe_id})
            MATCH (f1:Food {id: $food1_id})
            MATCH (f2:Food {id: $food2_id})
            WHERE id(f1) <> id(f2)
            MERGE (r)-[:CONTAINS]->(f1)
            MERGE (r)-[:CONTAINS]->(f2)
            MERGE (f1)-[:USED_TOGETHER]->(f2)
            """,
            recipe_id=recipe_id,
            food1_id=food1["node_id"],
            food2_id=food2["node_id"]
        )

In [23]:
# レシピデータもネットワーク化する
with driver.session() as session:
    session.execute_write(drop_recipe_data)
    for title, recipe in new_json_data.items():
        recipe_id = make_recipe(session, title, recipe)
        for index in range(0, len(recipe["ingredients"]) - 1, 2):
            food1 = find_food(session, recipe["ingredients"][index])
            food2 = find_food(session, recipe["ingredients"][index + 1])
            session.execute_write(make_food_network, recipe_id, food1, food2)
            

## 統計情報をあらかじめ追加

In [15]:
# レシピのページランクやAromaのページランクを取得する
def drop_projection_graph(tx, project_name):
    query = """
        CALL gds.graph.exists($project_name)
        YIELD exists
        WITH exists
        WHERE exists = true
        CALL gds.graph.drop($project_name) YIELD graphName
        RETURN graphName;
    """
    tx.run(query, project_name=project_name)
    
def create_aroma_page_rank(tx):
    query = """
    CALL gds.graph.project.cypher(
      'AromaPageRank',
      'MATCH (n) 
      WHERE n:Food OR n:Aroma
      RETURN id(n) AS id, labels(n) AS labels',
      'MATCH (n)-[r:SCENTED]-(m) 
      RETURN id(n) AS source, id(m) AS target, type(r) AS type',
      { validateRelationships: false }
    )
    """
    tx.run(query)
    query = """
        CALL gds.pageRank.stream('AromaPageRank')
        YIELD nodeId, score
        MATCH (n) WHERE id(n) = nodeId
        SET n.aroma_page_rank = score
        RETURN n.name, n.pagerank
        ORDER BY score DESC;
    """
    tx.run(query)

def create_recipe_page_rank(tx):
    query = """
    CALL gds.graph.project.cypher(
      'RecipePageRank',
      'MATCH (n) WHERE n:Recipe OR n:Food RETURN id(n) AS id, labels(n) AS labels',
      'MATCH (n)-[r:CONTAINS]-(m) RETURN id(n) AS source, id(m) AS target, type(r) AS type',
      { validateRelationships: false }
    )
    """
    tx.run(query)
    query = """
        CALL gds.pageRank.stream('RecipePageRank')
        YIELD nodeId, score
        MATCH (n) WHERE id(n) = nodeId
        SET n.recipe_page_rank = score
        RETURN n.name, n.pagerank
        ORDER BY score DESC;
    """
    tx.run(query)
    
def write_popularity(tx):
    query = """
        MATCH (f:Food)
        OPTIONAL MATCH (f)-[r:USED_TOGETHER]->(f2:Food)
        WITH f, COUNT(r) AS popularity
        SET f.popularity = popularity
    """
    tx.run(query)
    
def write_frequency(tx): 
    query2 = """
        MATCH (r:Recipe)-[:CONTAINS]->(f1:Food)-[rel:USED_TOGETHER]->(f2:Food)
        WHERE id(f1) <> id(f2)
        WITH f1, f2, rel, COUNT(DISTINCT r) AS frequency
        SET rel.frequency = frequency
    """
    tx.run(query2)
    
def write_shared_aromas(tx):
    query1 = """
        MATCH (f1:Food)-[:SCENTED]->(a:Aroma)<-[:SCENTED]-(f2:Food)
        WHERE id(f1) <> id(f2)
        WITH f1, f2, COUNT(a) AS shared_aromas
        SET f1.shared_aromas = shared_aromas
    """
    tx.run(query1)

with driver.session() as session:
    session.write_transaction(drop_projection_graph, "AromaPageRank")
    session.write_transaction(drop_projection_graph, "RecipePageRank")
    session.write_transaction(create_aroma_page_rank)
    session.write_transaction(write_popularity)
    session.write_transaction(write_frequency)
    session.write_transaction(create_recipe_page_rank)
    session.write_transaction(write_shared_aromas)

/var/folders/g4/fbyxhbrx6h3_4s4m15x3bly00000gn/T/ipykernel_33746/238045962.py:84: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(drop_projection_graph, "AromaPageRank")
/var/folders/g4/fbyxhbrx6h3_4s4m15x3bly00000gn/T/ipykernel_33746/238045962.py:85: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(drop_projection_graph, "RecipePageRank")
/var/folders/g4/fbyxhbrx6h3_4s4m15x3bly00000gn/T/ipykernel_33746/238045962.py:86: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_aroma_page_rank)
/var/folders/g4/fbyxhbrx6h3_4s4m15x3bly00000gn/T/ipykernel_33746/238045962.py:87: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(write_popularity)
/var/folders/g4/fbyxhbrx6h3_4s4m15x3bly00000gn/T/ipykernel_33746/238045962.py:88: DeprecationWarning: write_transaction has been renamed to execute_w